<a href="https://colab.research.google.com/github/FaisalWant/pandas_dataPipeline/blob/main/pandas_DataPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [3]:
import boto3 
import pandas as pd
from io import StringIO



In [ ]:
s3= boto3.resource('s3') 
bucket= s3.Bucket('deutsche-boerse-xetra-pds') 


In [ ]:
bucket_obj= bucket.objects.filter(Prefix='2021-03-15') 
objects= [obj for obj in bucket_obj]

In [ ]:
csv_obj_init= bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8') 
data= StringIO(csv_obj_init) 
df_init= pd.read_csv(data, delimiter=',')

In [ ]:
df_all= pd.DataFrame(columns=df_init.columns)

for obj in objects:  
  csv_obj= bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
  data= StringIO(csv_obj) 
  df= pd.read_csv(data, delimiter=",") 
  df_all = df_all.append(df, ignore_index=True)  

In [ ]:
columns=['date', 'month', 'year', 'day', 'name', 'open', 'close', 'high', 'low', 'volume']
df_all= df_all.loc[:, columns]

In [ ]:
data = StringIO(csv_obj) 
df= pd.read_csv(data, delimiter=',') 

In [2]:

# Using New DataSet 

import pandas as pd
tech_url= "https://andybek.com/pandas-tech"
tech= pd.read_csv(tech_url) 

In [3]:
# Filtering Columns 
tech.columns
columns=['date', 'month', 'year', 'day', 'name', 'open', 'close', 'high', 'low', 'volume']

df_all= tech.loc[:, columns]


df_all

,date,month,year,day,name,open,close,high,low,volume
0,2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622
1,2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851
2,2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719
3,2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745
4,2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...
7100,2019-08-23,8,2019,23,MSFT,137.19,133.39,138.35,132.80,38515386
7101,2019-08-23,8,2019,23,AAPL,209.43,202.64,212.05,201.00,46882843
7102,2019-08-23,8,2019,23,GOOGL,1185.17,1153.58,1195.67,1150.00,1813141
7103,2019-08-23,8,2019,23,AMZN,1793.03,1749.62,1804.90,1745.23,5277898


In [4]:
df_all.shape

(7105, 10)

In [8]:
df_all['opening_price']= df_all.sort_values(by=['year']).groupby(['name','date'])['open'].transform('first')

df_all

,date,month,year,day,name,open,close,high,low,volume,opening_price
0,2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,54.86
1,2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,79.38
2,2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,557.73
3,2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,37.35
4,2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,398.80
...,...,...,...,...,...,...,...,...,...,...,...
7100,2019-08-23,8,2019,23,MSFT,137.19,133.39,138.35,132.80,38515386,137.19
7101,2019-08-23,8,2019,23,AAPL,209.43,202.64,212.05,201.00,46882843,209.43
7102,2019-08-23,8,2019,23,GOOGL,1185.17,1153.58,1195.67,1150.00,1813141,1185.17
7103,2019-08-23,8,2019,23,AMZN,1793.03,1749.62,1804.90,1745.23,5277898,1793.03


In [ ]:
df_all= df_all.groupby(['year'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'close'), minimum_price_eur=('low', 'min'), maximum_price_eur=("high", "max"), daily_traded_volumn=('volume', 'sum'))